In [2]:
from google.colab import drive

# 구글 드라이브 마운트
drive.mount('/content/drive')

Mounted at /content/drive


## xml파일 하나로 테스트

In [1]:
import xml.etree.ElementTree as ET
import json

In [2]:
# XML 파일 경로
xml_file = "/content/drive/MyDrive/특허데이터/20190115_only_xml/1020170084262.xml"

In [3]:
# 파일을 열어서 내용 읽기 (인코딩 지정)
with open(xml_file, "r", encoding="EUC-KR") as file:  # "utf-8" 대신 실제 인코딩으로 변경
    xml_content = file.read()

In [4]:
# XML 문자열 파싱
root = ET.fromstring(xml_content)

## 필요한 데이터 추출

In [5]:
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

In [6]:
# 발명 Title추출
title_tag = root.find(".//KR_RequestInformation/KR_InventionTitle")

# Field, Background Art추출
description = root.find(".//description")
if description is not None:
    # Field 추출
    field_tag = description.find("technical-field")
    if field_tag is not None:
        field_paragraphs = [p.text.strip() for p in field_tag.findall("p") if p.text]
    # Background art추출
    background_art_tag = description.find("background-art")
    if background_art_tag is not None:
        background_paragraphs = [p.text.strip() for p in background_art_tag.findall("p") if p.text]

# Summary추출
summary_tag = root.find(".//abstract/summary")
if summary_tag is not None:
    # BeautifulSoup을 사용하여 <br/> 태그 처리
    soup = BeautifulSoup(ET.tostring(summary_tag, encoding="unicode"), "html.parser")

extracted_data = {
    "title": title_tag.text if title_tag is not None else None,
    "field": " ".join(field_paragraphs),
    "background_art": " ".join(background_paragraphs),
    "summary": soup.get_text(separator=" ").strip()
}

In [7]:
# KR_IPCInformation 내에서만 메인 분류와 서브 분류 검색
ipc_info = root.find(".//KR_Bibliographic/KR_IPCInformation")  # KR_IPCInformation 요소 찾기
main_class_data = {}
sub_class_data = []

if ipc_info is not None:  # KR_IPCInformation이 존재할 경우
    # 메인 분류 추출
    main_class = ipc_info.find("KR_MainClassification")
    if main_class is not None:
        full_code = main_class.text.split("\t")[0]  # 공백 기준으로 코드와 뒤에 정보 분리
        main_class_data = {
            "section": full_code[0],  # 첫 문자: 섹션 (예: F)
            "class": full_code[:3],  # 첫 3글자: 클래스 (예: F24)
            "subclass": full_code.split(" ")[0]  # 공백 전까지: 서브클래스 (예: F24F)
        }

    # 서브 분류 추출
    sub_classes = ipc_info.findall("KR_SubClassification")
    for sub_class in sub_classes:
        full_code = sub_class.text.split("\t")[0]
        sub_class_data.append({
            "section": full_code[0],  # 첫 문자: 섹션 (예: G)
            "class": full_code[:3],  # 첫 3글자: 클래스 (예: G08)
            "subclass": full_code.split(" ")[0]  # 공백 전까지: 서브클래스 (예: G08B)
        })


In [8]:
#디버깅
print("Title:", extracted_data["title"])
print("Field:", extracted_data["field"])
print("Background Art:", extracted_data["background_art"])
print("Summary:", extracted_data['summary'])
print("Main Classification:", main_class_data)
print("Sub Classifications:", sub_class_data)

Title: 이동형 공조장치
Field: 본 발명은 이동형 공조장치에 관한 것으로서, 보다 상세하게는, 다양한 기능들을 직관적이고 손쉽게 원격 제어할 수 있는 어플리케이션을 구비함으로써, 사용하기 매우 간편하고, 이를 통해, 사용자의 편의성을 개선시킬 수 있는 이동형 공조장치에 관한 것이다.
Background Art: 최근 들어, 다양한 이동형 공조장치가 개발되고 있다. 그 일례로서, 본 출원인이 출원한 특허출원 제2016-62734호의 로봇 공조장치, 특허출원 제2016-142990호의 이동형 공조장치 및 그 제어방법, 특허출원 제2016-113730호의 이동형 공조장치 등이 있다. 이 기술은, 공조장치 본체와, 공조장치 본체에 설치되며 냉,온풍을 발생시키는 공조부와, 공조부하를 감지하는 감지부와, 공조장치 본체를 이동시키는 이동부와, 감지부에서 감지된 공조부하측으로 공조부가 이동하여 냉,온풍을 송풍할 수 있도록 이동부와 공조부를 제어하는 제어부를 포함한다. 특히, 감지부는, 공조부하, 예를 들면, 사람을 감지하는 인체감지센서와, 사람주변의 공기조건, 예를 들면, 공기 온도와 습도와 이산화탄소 농도와 미세먼지 농도 등을 감지하는 각종 센서들을 포함한다. 제어부는, 감지부의 각 센서들로부터 입력된 감지 데이터를 처리하여, 최적의 제어값을 산출하고, 산출된 최적의 제어값을 기준으로 공조부를 제어한다. 따라서, 사람이 있는 실내부분에 최적의 온도와 풍량과 습도를 갖는 공기를 송풍할 수 있게 한다. 이로써, 사람이 있는 실내부분을 쾌적하게 유지시킨다. 여기서, 제어부는, 원격제어 단말기, 예를 들면, 스마트폰과 무선통신이 가능하고, 이러한 무선통신을 통해 원격제어가 가능하다. 특히, 스마트폰으로 이동형 공조장치의 공조부를 제어하여 송풍 공기의 온도와 풍량 등을 조절하고, 스마트폰으로 이동형 공조장치를 호출하면, 호출한 스마트폰측으로 공조장치 본체를 이동시켜 호출된 곳을 냉,난방하며, 스마트폰으로 냉,난방이 필요한 곳을 지정하면, 지정된 곳으로 공조장치 본체를 이동

## JSON구조 완성

In [9]:
#JSON구조 생성
combined_data={
    "title":extracted_data["title"],
    "field":extracted_data["field"],
    "background_art":extracted_data["background_art"],
    "summary": extracted_data["summary"],
    "main_classification":main_class_data,
    "sub_classification":sub_class_data
}


In [11]:
print(combined_data)

{'title': '이동형 공조장치', 'field': '본 발명은 이동형 공조장치에 관한 것으로서, 보다 상세하게는, 다양한 기능들을 직관적이고 손쉽게 원격 제어할 수 있는 어플리케이션을 구비함으로써, 사용하기 매우 간편하고, 이를 통해, 사용자의 편의성을 개선시킬 수 있는 이동형 공조장치에 관한 것이다.', 'background_art': '최근 들어, 다양한 이동형 공조장치가 개발되고 있다. 그 일례로서, 본 출원인이 출원한 특허출원 제2016-62734호의 로봇 공조장치, 특허출원 제2016-142990호의 이동형 공조장치 및 그 제어방법, 특허출원 제2016-113730호의 이동형 공조장치 등이 있다. 이 기술은, 공조장치 본체와, 공조장치 본체에 설치되며 냉,온풍을 발생시키는 공조부와, 공조부하를 감지하는 감지부와, 공조장치 본체를 이동시키는 이동부와, 감지부에서 감지된 공조부하측으로 공조부가 이동하여 냉,온풍을 송풍할 수 있도록 이동부와 공조부를 제어하는 제어부를 포함한다. 특히, 감지부는, 공조부하, 예를 들면, 사람을 감지하는 인체감지센서와, 사람주변의 공기조건, 예를 들면, 공기 온도와 습도와 이산화탄소 농도와 미세먼지 농도 등을 감지하는 각종 센서들을 포함한다. 제어부는, 감지부의 각 센서들로부터 입력된 감지 데이터를 처리하여, 최적의 제어값을 산출하고, 산출된 최적의 제어값을 기준으로 공조부를 제어한다. 따라서, 사람이 있는 실내부분에 최적의 온도와 풍량과 습도를 갖는 공기를 송풍할 수 있게 한다. 이로써, 사람이 있는 실내부분을 쾌적하게 유지시킨다. 여기서, 제어부는, 원격제어 단말기, 예를 들면, 스마트폰과 무선통신이 가능하고, 이러한 무선통신을 통해 원격제어가 가능하다. 특히, 스마트폰으로 이동형 공조장치의 공조부를 제어하여 송풍 공기의 온도와 풍량 등을 조절하고, 스마트폰으로 이동형 공조장치를 호출하면, 호출한 스마트폰측으로 공조장치 본체를 이동시켜 호출된 곳을 냉,난방하며, 스마트폰으로 냉,난방이 필요한 곳을 지정하면, 지정된 곳

## JSON 파일 저장

In [10]:
# JSON 파일로 저장
output_file = "ipc_data_v1.0.json"
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(combined_data, json_file, ensure_ascii=False, indent=4)

print(f"JSON 파일이 저장되었습니다: {output_file}")

JSON 파일이 저장되었습니다: ipc_data_v1.0.json
